In [13]:
import psycopg2
import json

# Load JSON data from the file with UTF-8 encoding
with open(r'C:\Users\Jimmy\Desktop\final-project\en.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="pokemontcg",
    user="jliao",
    password="D3c0y0ht4n1!",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Clear the existing data in the pokemon_sets table
cur.execute("DELETE FROM pokemon_sets")

# Insert data into the table with flattened legalities
insert_query = """
INSERT INTO pokemon_sets (
    id, name, series, printed_total, total, legality_unlimited, legality_expanded, 
    legality_standard, ptcgo_code, release_date, updated_at, symbol_url, logo_url
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Loop through the JSON data and insert each item
for item in data:
    # Flatten legalities dictionary
    legalities = item.get('legalities', {})
    legality_unlimited = legalities.get('unlimited', None)
    legality_expanded = legalities.get('expanded', None)
    legality_standard = legalities.get('standard', None)
    
    cur.execute(insert_query, (
        item.get('id'),
        item.get('name'),
        item.get('series'),
        item.get('printedTotal', None),
        item.get('total', None),
        legality_unlimited,
        legality_expanded,
        legality_standard,
        item.get('ptcgoCode', None),
        item.get('releaseDate', None),
        item.get('updatedAt', None),
        item.get('images', {}).get('symbol', None),
        item.get('images', {}).get('logo', None)
    ))

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

print("Data reloaded successfully!")


Data reloaded successfully!


In [15]:
import psycopg2
import json
import os

# Directory containing the JSON files
json_directory = r'C:\Users\Jimmy\Desktop\final-project\cards\en'

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="pokemontcg",
    user="jliao",
    password="D3c0y0ht4n1!",
    host="localhost",
    port="5432"
)

# Create the cursor object here, before the loop
cursor = conn.cursor()

# Fetch all IDs from pokemon_sets
cursor.execute("SELECT id FROM pokemon_sets;")
pokemon_set_ids = cursor.fetchall()

# Define the common table structure with flattened fields
create_table_template = """
CREATE TABLE IF NOT EXISTS "{table_id}" (
    card_id SERIAL PRIMARY KEY,
    card_name VARCHAR(255),
    card_supertype VARCHAR(255),
    card_subtypes VARCHAR[],
    card_hp INT,
    card_types VARCHAR[],
    card_evolvesTo VARCHAR[],
    card_evolvesFrom VARCHAR(255),
    card_attacks_names VARCHAR[],
    card_attacks_costs VARCHAR[],
    card_attacks_damage VARCHAR[],
    card_attacks_texts VARCHAR[],
    card_weaknesses_types VARCHAR[],
    card_weaknesses_values VARCHAR[],
    card_resistances_types VARCHAR[],
    card_resistances_values VARCHAR[],
    card_retreatCost VARCHAR[],
    card_convertedRetreatCost INT,
    card_flavorText TEXT,
    card_number VARCHAR(50),
    card_artist VARCHAR(255),
    card_rarity VARCHAR(50),
    card_nationalPokedexNumbers INT[],
    card_legalities_unlimited BOOLEAN,
    card_legalities_expanded BOOLEAN,
    card_image_small VARCHAR(255),
    card_image_large VARCHAR(255)
);
"""

# Get the list of JSON files from the directory
json_files = [file for file in os.listdir(json_directory) if file.endswith('.json')]

# Helper function to handle missing values
def handle_missing(value, value_type="str"):
    if value is None:
        if value_type == "int" or value_type == "float":
            return 0
        else:
            return None
    return value

# Ensure that each JSON file corresponds to a table in pokemon_sets
for (pokemon_set_id,) in pokemon_set_ids:
    corresponding_json_file = f"{pokemon_set_id}.json"
    
    if corresponding_json_file in json_files:
        file_path = os.path.join(json_directory, corresponding_json_file)
        
        # Load the JSON data with explicit UTF-8 encoding
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Create the table for the corresponding pokemon_set_id
        create_table_sql = create_table_template.format(table_id=pokemon_set_id)
        cursor.execute(create_table_sql)

        # Loop through each card in the JSON file
        for card in data:
            # Prepare attacks, weaknesses, resistances as flattened lists
            attack_names = [attack.get('name') for attack in card.get('attacks', [])]
            attack_costs = [", ".join(attack.get('cost', [])) for attack in card.get('attacks', [])]
            attack_damage = [attack.get('damage', '0') for attack in card.get('attacks', [])]
            attack_texts = [attack.get('text', '') for attack in card.get('attacks', [])]

            weakness_types = [weakness.get('type') for weakness in card.get('weaknesses', [])]
            weakness_values = [weakness.get('value') for weakness in card.get('weaknesses', [])]

            resistance_types = [resistance.get('type') for resistance in card.get('resistances', [])]
            resistance_values = [resistance.get('value') for resistance in card.get('resistances', [])]

            # Ensure nationalPokedexNumbers is handled as an array of integers
            national_pokedex_numbers = card.get('nationalPokedexNumbers', [])
            if isinstance(national_pokedex_numbers, int):
                national_pokedex_numbers = [national_pokedex_numbers]

            # Insert the card's data into the table
            insert_sql = f"""
            INSERT INTO "{pokemon_set_id}" (
                card_name, card_supertype, card_subtypes, card_hp, card_types, card_evolvesTo, card_evolvesFrom, 
                card_attacks_names, card_attacks_costs, card_attacks_damage, card_attacks_texts, 
                card_weaknesses_types, card_weaknesses_values, card_resistances_types, card_resistances_values, 
                card_retreatCost, card_convertedRetreatCost, card_flavorText, card_number, card_artist, card_rarity, 
                card_nationalPokedexNumbers, card_legalities_unlimited, card_legalities_expanded, card_image_small, card_image_large
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            
            cursor.execute(insert_sql, (
                handle_missing(card.get('name')),
                handle_missing(card.get('supertype')),
                handle_missing(card.get('subtypes')),
                handle_missing(card.get('hp'), "int"),
                handle_missing(card.get('types')),
                handle_missing(card.get('evolvesTo')),
                handle_missing(card.get('evolvesFrom')),
                attack_names,
                attack_costs,
                attack_damage,
                attack_texts,
                weakness_types,
                weakness_values,
                resistance_types,
                resistance_values,
                handle_missing(card.get('retreatCost')),
                handle_missing(card.get('convertedRetreatCost'), "int"),
                handle_missing(card.get('flavorText')),
                handle_missing(card.get('number')),
                handle_missing(card.get('artist')),
                handle_missing(card.get('rarity')),
                national_pokedex_numbers,
                card.get('legalities', {}).get('unlimited') == 'Legal',
                card.get('legalities', {}).get('expanded') == 'Legal',
                handle_missing(card.get('images', {}).get('small')),
                handle_missing(card.get('images', {}).get('large'))
            ))

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("Tables created and card data inserted successfully!")


Tables created and card data inserted successfully!
